# Setup the Env

In [1]:
import sys
import os

import sagemaker
from sagemaker import get_execution_role

# Add the parent directory to the sys.path
sys.path.insert(0, os.path.abspath('..'))

# Define IAM role
role = get_execution_role()
role

# Establish S3 bucket connection
import boto3
s3 = boto3.client('s3')
bucket = 'capstone-bucket-4-friends'

# Take a look at current dir
print(os.getcwd())

from file_utilities import s3_download 

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
/home/sagemaker-user/capstone-2024-summer/src/rachel


# Import libraries

In [2]:
# standard libraries
import numpy as np
import pandas as pd

# visualization
import matplotlib.pyplot as plt
!pip install seaborn -q
import seaborn as sns

# Download Data from S3 as needed

In [3]:
#crsp = "CRSP/crsp_2018-2023.csv"
#crsp_path = s3_download(crsp)

In [4]:
#index = "Index/security_master.csv"
#index_path = s3_download(index)

# Load the dataset

https://www.crsp.org/wp-content/uploads/guides/CRSP_US_Stock_&_Indexes_Database_Data_Descriptions_Guide.pdf

In [5]:
dtype_dict = {
    'permno_id': 'object',
    'ncusip_id': 'object',
    'cusip_id': 'object',
    'ticker': 'object',
    'share_code': 'object',
    'share_code_type': 'object',
    'share_code_detail': 'object',
    'exchange_code': 'object',
    'company_name': 'object',
    'primary_exchange': 'object',
    'trading_status': 'object',
    'security_status': 'object',
    'naics': 'object',
    'naics_sector': 'object',
    'naics_secondary': 'object',
    'ask_or_high_price': 'float64',
    'ask_price': 'float64',
    'bid_or_low_price': 'float64',
    'bid_price': 'float64',
    'open_price': 'float64',
    'close_price': 'float64',
    'return': 'float64',
    'volume': 'int64',
    'shares_outstanding': 'int64',
    'num_trades': 'int64',
    'factor_to_adjust_price': 'float64',
    'factor_to_adjust_shares': 'float64',
    'value_weighted_return': 'float64',
    'equal_weighted_return': 'float64',
    'return_on_SP_index': 'float64',
    'log_return': 'float64',
    'market_cap': 'float64',
    'weighted_price': 'float64',
    'sector_weighted_avg_log_return': 'float64',
    'sector_simple_avg_log_return': 'float64',
    'weighted_avg_log_return': 'float64',
    'simple_avg_log_return': 'float64'
}

In [6]:
crsp_df = pd.read_csv("/home/sagemaker-user/capstone-2024-summer/data/crsp_2018-2023_clean_2.csv", dtype=dtype_dict, parse_dates=["date"])

crsp_df.head()

,date,permno_id,ncusip_id,cusip_id,ticker,share_code,share_code_type,share_code_detail,exchange_code,company_name,...,value_weighted_return,equal_weighted_return,return_on_SP_index,log_return,market_cap,weighted_price,sector_weighted_avg_log_return,sector_simple_avg_log_return,weighted_avg_log_return,simple_avg_log_return
0,2018-01-02,10026,46603210,46603210,JJSF,11.0,1,1,3,J & J SNACK FOODS CORP,...,0.008505,0.011232,0.008303,-0.017608,2.784892e+06,4.154502e+08,0.000000,0.000000,-0.0,-0.00064
1,2018-01-03,10026,46603210,46603210,JJSF,11.0,1,1,3,J & J SNACK FOODS CORP,...,0.005856,0.005103,0.006399,-0.010038,2.757077e+06,4.071927e+08,0.009281,-0.003229,-0.0,-0.00064
2,2018-01-04,10026,46603210,46603210,JJSF,11.0,1,1,3,J & J SNACK FOODS CORP,...,0.004001,0.005272,0.004029,0.013718,2.795160e+06,4.185193e+08,0.000556,0.006031,0.0,-0.00064
3,2018-01-05,10026,46603210,46603210,JJSF,11.0,1,1,3,J & J SNACK FOODS CORP,...,0.005804,0.003880,0.007034,-0.009596,2.768464e+06,4.105633e+08,0.008654,0.005565,-0.0,-0.00064
4,2018-01-08,10026,46603210,46603210,JJSF,11.0,1,1,3,J & J SNACK FOODS CORP,...,0.001816,0.000584,0.001662,0.000742,2.770518e+06,4.111726e+08,0.004083,0.005570,0.0,-0.00064


In [7]:
sp500_df = pd.read_csv("/home/sagemaker-user/capstone-2024-summer/data/security_master.csv")

sp500_df.head()

,cusip,PERMNO,PERMCO,HSHRCD,DLSTCD,HTICK,HCOMNAM,HTSYMBOL,HNAICS,HPRIMEXC,...,NUMDEL,NUMNDI,BEGDAT,ENDDAT,BEGPRC,ENDPRC,BEGRET,ENDRET,BEGVOL,ENDVOL
0,00130H10,76712,10996,11,100,AES,A E S CORP,AES,221118,N,...,1,68,06/26/1991,12/29/2023,06/26/1991,12/29/2023,06/26/1991,12/29/2023,06/26/1991,12/29/2023
1,00206R10,66093,21645,11,100,T,A T & T INC,T,517312,N,...,1,0,02/16/1984,12/29/2023,02/16/1984,12/29/2023,02/16/1984,12/29/2023,02/16/1984,12/29/2023
2,00507V10,79678,12499,11,233,NaN,ACTIVISION BLIZZARD INC,ATVI,513210,Q,...,1,1968,10/22/1993,10/12/2023,10/22/1993,10/12/2023,10/22/1993,10/12/2023,10/22/1993,10/12/2023
3,00724F10,75510,8476,11,100,ADBE,ADOBE INC,ADBE,511210,Q,...,1,2185,08/13/1986,12/29/2023,08/13/1986,12/29/2023,08/13/1986,12/29/2023,08/13/1986,12/29/2023
4,00971T10,87299,17300,11,100,AKAM,AKAMAI TECHNOLOGIES INC,AKAM,511210,Q,...,1,1722,10/29/1999,12/29/2023,10/29/1999,12/29/2023,10/29/1999,12/29/2023,10/29/1999,12/29/2023


# Take a look at the columns

In [8]:
crsp_df.columns

Index(['date', 'permno_id', 'ncusip_id', 'cusip_id', 'ticker', 'share_code',
       'share_code_type', 'share_code_detail', 'exchange_code', 'company_name',
       'primary_exchange', 'trading_status', 'security_status', 'naics',
       'naics_sector', 'naics_secondary', 'ask_or_high_price', 'ask_price',
       'bid_or_low_price', 'bid_price', 'open_price', 'close_price', 'return',
       'volume', 'shares_outstanding', 'num_trades', 'factor_to_adjust_price',
       'factor_to_adjust_shares', 'value_weighted_return',
       'equal_weighted_return', 'return_on_SP_index', 'log_return',
       'market_cap', 'weighted_price', 'sector_weighted_avg_log_return',
       'sector_simple_avg_log_return', 'weighted_avg_log_return',
       'simple_avg_log_return'],
      dtype='object')

In [9]:
crsp_df.dtypes

date                              datetime64[ns]
permno_id                                 object
ncusip_id                                 object
cusip_id                                  object
ticker                                    object
share_code                                object
share_code_type                           object
share_code_detail                         object
exchange_code                             object
company_name                              object
primary_exchange                          object
trading_status                            object
security_status                           object
naics                                     object
naics_sector                              object
naics_secondary                           object
ask_or_high_price                        float64
ask_price                                float64
bid_or_low_price                         float64
bid_price                                float64
open_price          

In [10]:
og_crsp_len = len(crsp_df)
og_crsp_cols = len(crsp_df.columns)
print("number of records:", og_crsp_len)
print("number of columns:", og_crsp_cols)

number of records: 7658922
number of columns: 38


## A. Keep only PERMNO in S&P500

In [11]:
PERMNO = sp500_df["PERMNO"].astype(str).unique()
len(PERMNO)

594

In [12]:
og_len = len(crsp_df)
crsp_df = crsp_df[crsp_df["permno_id"].isin(PERMNO)]
new_len = len(crsp_df)
print("num_records_removed: ", og_len - new_len)
print("num_records_kept: ", new_len)

num_records_removed:  6849039
num_records_kept:  809883


In [13]:
len(crsp_df["permno_id"].unique())

554

# Check a few large companies to ensure they are still in the dataset

["GOOG", "NVDA", "TSLA", "AMZN"]

In [14]:
assert all(ticker in crsp_df["ticker"].unique() for ticker in ["GOOG", "NVDA", "TSLA", "AMZN"])

# Take a final look at the columns

In [15]:
crsp_df.columns

Index(['date', 'permno_id', 'ncusip_id', 'cusip_id', 'ticker', 'share_code',
       'share_code_type', 'share_code_detail', 'exchange_code', 'company_name',
       'primary_exchange', 'trading_status', 'security_status', 'naics',
       'naics_sector', 'naics_secondary', 'ask_or_high_price', 'ask_price',
       'bid_or_low_price', 'bid_price', 'open_price', 'close_price', 'return',
       'volume', 'shares_outstanding', 'num_trades', 'factor_to_adjust_price',
       'factor_to_adjust_shares', 'value_weighted_return',
       'equal_weighted_return', 'return_on_SP_index', 'log_return',
       'market_cap', 'weighted_price', 'sector_weighted_avg_log_return',
       'sector_simple_avg_log_return', 'weighted_avg_log_return',
       'simple_avg_log_return'],
      dtype='object')

In [16]:
crsp_df.dtypes

date                              datetime64[ns]
permno_id                                 object
ncusip_id                                 object
cusip_id                                  object
ticker                                    object
share_code                                object
share_code_type                           object
share_code_detail                         object
exchange_code                             object
company_name                              object
primary_exchange                          object
trading_status                            object
security_status                           object
naics                                     object
naics_sector                              object
naics_secondary                           object
ask_or_high_price                        float64
ask_price                                float64
bid_or_low_price                         float64
bid_price                                float64
open_price          

In [17]:
crsp_len = len(crsp_df)
crsp_cols = len(crsp_df.columns)
print("number of records:", crsp_len)
print("number of columns:", crsp_cols)

number of records: 809883
number of columns: 38


# Save the cleaned df as csv & parquet

In [18]:
crsp_df.to_csv("/home/sagemaker-user/capstone-2024-summer/data/crsp_2018-2023_clean_3.csv", index=False)
crsp_df.to_parquet("/home/sagemaker-user/capstone-2024-summer/data/crsp_2018-2023_clean_3.parquet")

In [19]:
reload = pd.read_parquet("/home/sagemaker-user/capstone-2024-summer/data/crsp_2018-2023_clean_3.parquet")
assert crsp_df.equals(reload)

reload.head()

,date,permno_id,ncusip_id,cusip_id,ticker,share_code,share_code_type,share_code_detail,exchange_code,company_name,...,value_weighted_return,equal_weighted_return,return_on_SP_index,log_return,market_cap,weighted_price,sector_weighted_avg_log_return,sector_simple_avg_log_return,weighted_avg_log_return,simple_avg_log_return
9263,2018-01-02,10104,68389X10,68389X10,ORCL,11.0,1,1,1,ORACLE CORP,...,0.008505,0.011232,0.008303,-0.013843,1.930296e+08,9.000972e+09,0.000000,0.000000,-0.0,-0.00064
9264,2018-01-03,10104,68389X10,68389X10,ORCL,11.0,1,1,1,ORACLE CORP,...,0.005856,0.005103,0.006399,0.022897,1.975004e+08,9.422745e+09,0.021973,0.005694,0.0,-0.00064
9265,2018-01-04,10104,68389X10,68389X10,ORCL,11.0,1,1,1,ORACLE CORP,...,0.004001,0.005272,0.004029,0.009803,1.994460e+08,9.609309e+09,0.002940,0.001681,0.0,-0.00064
9266,2018-01-05,10104,68389X10,68389X10,ORCL,11.0,1,1,1,ORACLE CORP,...,0.005804,0.003880,0.007034,0.006001,2.006465e+08,9.725336e+09,0.015718,0.004005,0.0,-0.00064
9267,2018-01-08,10104,68389X10,68389X10,ORCL,11.0,1,1,1,ORACLE CORP,...,0.001816,0.000584,0.001662,0.010467,2.027577e+08,9.931072e+09,0.006267,0.009789,0.0,-0.00064


# Upload the parquet to s3

In [20]:
s3.upload_file("/home/sagemaker-user/capstone-2024-summer/data/crsp_2018-2023_clean_3.parquet", bucket, "CRSP/crsp_2018-2023_clean_3.parquet")

In [21]:
reload_path = s3_download("CRSP/crsp_2018-2023_clean_3.parquet")

In [22]:
reload = pd.read_parquet("/home/sagemaker-user/capstone-2024-summer/data/crsp_2018-2023_clean_3.parquet")
assert crsp_df.equals(reload)

reload.head()

,date,permno_id,ncusip_id,cusip_id,ticker,share_code,share_code_type,share_code_detail,exchange_code,company_name,...,value_weighted_return,equal_weighted_return,return_on_SP_index,log_return,market_cap,weighted_price,sector_weighted_avg_log_return,sector_simple_avg_log_return,weighted_avg_log_return,simple_avg_log_return
9263,2018-01-02,10104,68389X10,68389X10,ORCL,11.0,1,1,1,ORACLE CORP,...,0.008505,0.011232,0.008303,-0.013843,1.930296e+08,9.000972e+09,0.000000,0.000000,-0.0,-0.00064
9264,2018-01-03,10104,68389X10,68389X10,ORCL,11.0,1,1,1,ORACLE CORP,...,0.005856,0.005103,0.006399,0.022897,1.975004e+08,9.422745e+09,0.021973,0.005694,0.0,-0.00064
9265,2018-01-04,10104,68389X10,68389X10,ORCL,11.0,1,1,1,ORACLE CORP,...,0.004001,0.005272,0.004029,0.009803,1.994460e+08,9.609309e+09,0.002940,0.001681,0.0,-0.00064
9266,2018-01-05,10104,68389X10,68389X10,ORCL,11.0,1,1,1,ORACLE CORP,...,0.005804,0.003880,0.007034,0.006001,2.006465e+08,9.725336e+09,0.015718,0.004005,0.0,-0.00064
9267,2018-01-08,10104,68389X10,68389X10,ORCL,11.0,1,1,1,ORACLE CORP,...,0.001816,0.000584,0.001662,0.010467,2.027577e+08,9.931072e+09,0.006267,0.009789,0.0,-0.00064
